In [1]:
library(keras)
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.0       ✔ purrr   0.3.2  
✔ tibble  2.1.1       ✔ dplyr   0.8.0.1
✔ tidyr   0.8.3       ✔ stringr 1.4.0  
✔ readr   1.3.1       ✔ forcats 0.4.0  
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
use_python("/home/kien/.pyenv/versions/miniconda3-4.3.30/envs/petfinder/bin/python")

In [3]:
data_dir <- file.path("..", "data")

In [4]:
train <- data.table::fread(file.path(data_dir, "train.csv"), data.table = FALSE)

In [5]:
img_size <- 256
img_channel <- 3
img_dim <- c(img_size, img_size, img_channel)
batch_size <- 16

In [6]:
load_image <- function(file) {
    image_load(file, target_size = c(img_size, img_size)) %>%
    image_to_array
}

In [7]:
base_layer <- application_densenet121(weights = "imagenet",
                                      input_shape = c(256, 256, 3),
                                      include_top = FALSE)

In [8]:
inputs = layer_input(c(256, 256, 3))

In [9]:
outputs = inputs %>%
    base_layer %>%
    layer_global_average_pooling_2d() %>%
    layer_lambda(k_expand_dims) %>%
    layer_average_pooling_1d(4) %>%
    layer_lambda(function(.) .[, , 1])

In [10]:
model <- keras_model(inputs, outputs)

In [11]:
image_features <- train %>%
    select(PetID) %>%
    mutate(img_file = sprintf("%s-1.jpg", PetID)) %>%
    mutate(img_path = file.path(data_dir, "train_images", img_file)) %>%
    mutate(img_feature = map(img_path, function(path) {
        tryCatch(
            path %>%
                load_image %>%
                array_reshape(c(1, img_dim)) %>%
                predict(model, .),
            error = function(e) { print(e); NULL }
        )
    })) %>%
    select(PetID, img_feature)

head(image_features)

<Rcpp::exception in py_call_impl(callable, dots$args, dots$keywords): FileNotFoundError: [Errno 2] No such file or directory: '../data/train_images/1bc0f89d8-1.jpg'

Detailed traceback: 
  File "/home/kien/.pyenv/versions/miniconda3-4.3.30/envs/petfinder/lib/python3.6/site-packages/keras_preprocessing/image/utils.py", line 104, in load_img
    img = pil_image.open(path)
  File "/home/kien/.pyenv/versions/miniconda3-4.3.30/envs/petfinder/lib/python3.6/site-packages/PIL/Image.py", line 2634, in open
    fp = builtins.open(filename, "rb")
>
<Rcpp::exception in py_call_impl(callable, dots$args, dots$keywords): FileNotFoundError: [Errno 2] No such file or directory: '../data/train_images/1bf24d8be-1.jpg'

Detailed traceback: 
  File "/home/kien/.pyenv/versions/miniconda3-4.3.30/envs/petfinder/lib/python3.6/site-packages/keras_preprocessing/image/utils.py", line 104, in load_img
    img = pil_image.open(path)
  File "/home/kien/.pyenv/versions/miniconda3-4.3.30/envs/petfinder/lib/python3.6/s

PetID,img_feature
86e1089a3,"3.570092e-03, 2.346859e-01, 8.743189e-02, 1.580948e-02, 1.433365e-01, 1.534586e-02, 1.634745e-03, 2.903937e-03, 2.851898e-02, 1.103072e-01, 1.265028e+00, 5.068045e-03, 1.932355e-02, 5.275663e-03, 1.121880e-01, 2.582251e-02, 1.471721e-03, 1.940461e-03, 3.012792e-02, 7.671193e-03, 3.973945e-02, 5.079819e-01, 1.349608e+00, 2.346137e-02, 1.917496e-01, 1.592725e-01, 1.186002e-01, 4.835925e-03, 9.611670e-02, 1.931198e-02, 1.493282e-02, 5.512129e-01, 4.112780e-03, 2.062314e-01, 2.728555e+00, 7.398576e-01, 8.677861e-03, 1.781196e-01, 8.376272e-01, 2.529628e-02, 1.716648e+00, 5.009141e-02, 2.729096e-01, 1.445808e-04, 2.309688e-02, 3.638843e-02, 1.029654e-02, 7.513016e-02, 8.097152e-03, 3.059697e-02, 2.901444e-01, 1.283195e-01, 1.353206e+00, 4.894236e-02, 1.072599e-03, 6.722973e-03, 5.679862e-03, 1.986208e-02, 2.730767e+00, 1.928083e-02, 6.340417e-02, 2.124390e-03, 2.250243e-04, 3.443983e-02, 3.315622e-01, 4.949830e-02, 6.808817e-03, 1.682718e-02, 1.621168e-03, 1.429126e-01, 1.438124e-02, 5.571822e-01, 8.197558e-03, 9.400913e-03, 7.831342e-03, 7.617285e-01, 1.111853e+00, 3.679382e-01, 1.659083e-02, 8.522930e-01, 1.834452e-02, 1.040010e-02, 3.809403e-02, 3.230619e-01, 2.968942e-02, 9.133589e-03, 9.673984e-02, 2.676135e-01, 1.203470e-01, 3.758113e-02, 1.183204e-01, 2.597102e-02, 2.071302e-01, 2.713950e-03, 6.832764e-01, 7.369562e-03, 2.319386e+00, 1.639312e-02, 5.326354e-03, 5.556299e-03, 2.122135e-02, 1.529735e-03, 6.361555e-01, 4.341408e-02, 7.260774e-03, 4.302610e-02, 4.437482e+00, 2.999458e-03, 4.730479e-01, 1.099278e+01, 1.644751e-02, 1.296502e-01, 6.541380e+00, 1.205165e-01, 3.812600e-02, 8.174029e-01, 1.852228e-02, 1.107274e-03, 3.782846e-01, 5.962559e-03, 5.013957e-02, 4.149576e-03, 1.604355e-03, 9.498929e-03, 9.047301e-02, 5.237416e-02, 2.381030e+00, 5.992968e-02, 3.545946e+00, 6.722967e+00, 2.919861e+00, 1.606593e+00, 2.322585e+00, 5.323454e+00, 1.122151e+00, 2.444414e+00, 1.573465e+00, 2.402645e+00, 6.606967e+00, 1.025083e+01, 8.262869e+00, 1.334037e+00, 9.305857e+00, 4.072010e+00, 5.472231e+00, 2.640587e+00, 8.119669e+00, 4.184534e+00, 6.283208e+00, 1.483286e+01, 9.307190e-01, 2.349127e+00, 2.879691e+00, 4.307833e+00, 3.452402e+00, 5.590841e+00, 6.602248e+00, 7.440899e-01, 1.559836e+00, 7.211305e-01, 3.197587e+00, 1.733037e-01, 3.810088e+00, 6.654818e+00, 4.640333e+00, 3.966069e+00, 7.463632e+00, 3.674305e+00, 8.125097e-01, 2.532910e+00, 4.165110e+00, 2.990715e+00, 1.859628e+01, 3.479825e+00, 9.368723e-01, 2.622565e+00, 3.916210e+00, 2.690929e+00, 2.887359e+00, 1.253260e+00, 2.668852e+00, 2.291803e+00, 4.229059e+00, 1.947396e+00, 3.016208e+00, 9.553901e+00, 2.661875e+00, 6.915376e+00, 5.743982e+00, 6.065076e+00, 2.264753e+00, 1.038917e+00, 2.504269e+00, 5.424947e+00, 3.455694e+00, 1.445556e+00, 1.222952e+00, 3.688951e+00, 3.794439e+00, 4.356835e+00, 9.452110e+00, 3.550905e+00, 5.512162e+00, 4.784231e+00, 2.406044e+00, 4.434781e+00, 6.771077e+00, 1.259927e+00, 4.831836e+00, 5.988461e+00, 2.908355e-01, 2.154951e+00, 2.672534e+00, 1.245131e+00, 1.747303e+00, 6.572837e+00, 6.580818e+00, 8.222555e+00, 1.409564e+00, 1.839717e+00, 3.722663e+00, 1.189891e+00, 5.365520e+00, 2.461152e+00, 3.235493e+00, 1.884957e+00, 1.431205e+00, 1.384764e+00, 3.911227e-01, 1.267311e+00, 3.401793e+00, 7.788378e+00, 1.998792e+00, 1.618217e+00, 4.666241e+00, 5.682925e+00, 1.220192e+00, 2.408618e+00, 4.386104e+00, 1.213854e+00, 1.493560e+00, 2.809523e+00, 2.850410e+00, 9.054415e-01, 2.990314e+00, 2.076530e+00, 3.325446e+00, 5.646461e+00, 1.265070e+00, 1.194027e+00, 3.830580e+00, 3.214630e+00, 1.703124e+00, 1.341653e+00, 2.304122e+00, 3.083212e+00"
6296e909a,"7.412266e-03, 1.629281e-01, 1.984409e-02, 7.903556e-02, 9.195366e-02, 1.829909e-02, 6.345603e-03, 7.113200e-03, 1.666708e-02, 7.790183e-02, 2.124274e+00, 1.165184e-03, 8.063426e-03, 2.539676e-03, 3.084868e-02, 6.362621e-02, 9.971091e-04, 3.645329e-03, 1.926881e-02, 2.747235e-03, 5.233019e-02, 1.171898e+00, 8.328310e-01, 1.654593e-02, 2.101616e+00, 3.464878e-01, 1.101966e+00

In [13]:
saveRDS(image_features, file.path(data_dir, "img_features.rds"))